In [16]:
import numpy as np
import cv2 as cv2
import datetime
from cv2 import aruco
import yaml
import os

In [17]:
# Define ARUCO dictionary to detect
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
parameters = cv2.aruco.DetectorParameters()

#Initialize the video capture (1 for computer camera)
cap = cv2.VideoCapture(0)

# Load calibration data which was previously done with aruco markers
with open('calibration.yaml', 'r') as f:
    calibration_data = yaml.safe_load(f)
mtx = np.array(calibration_data['camera_matrix'])
dist = np.array(calibration_data['dist_coeff'])

# Get the optimal camera matrix
ret, img = cap.read()
if not ret:
    print("Error: Could not read from the camera.")
    exit()
h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))


while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # GrayScale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect ArUco markers 1,2,3,4: corners   \   5: end    \     9:start
    corners, ids, rejected = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)


    # if 4 corner markers detected
    if ids is not None:
        cv2.aruco.drawDetectedMarkers(frame,None)

        # Extract corner markers
        marker_positions = {}
        for i, marker_id in enumerate(ids.flatten()):
            marker_positions[marker_id] = corners[i][0] 

        # Check if all four required markers are detected
        required_ids = {1, 2, 3, 4}
        if required_ids.issubset(marker_positions.keys()):
            rect_corners = np.array([
                marker_positions[1][0],  # Top-left corner
                marker_positions[2][1],  # Top-right corner
                marker_positions[3][2],  # Bottom-right corner
                marker_positions[4][3],  # Bottom-left corner
            ], dtype=np.float32)

            # Ensure rect_corners has the correct shape
            if rect_corners.shape == (4, 2):
                # Draw the rectangle around the detected markers
                cv2.polylines(frame, [np.int32(rect_corners)], isClosed=True, color=(0, 0, 255), thickness=3)

    # Show frame with rectangle
    cv2.imshow('ArUco Marker Detection', frame)

    #exit: 'q'
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):  # Quit if 'q' is pressed
        break

    #map: "m"
    elif key == ord('m') and 'rect_corners' in locals() and rect_corners.shape == (4, 2):
        # Define destination points for the flattened rectangle
        width, height = 500, 1000  # Desired output size
        dst_corners = np.array([
            [0, 0],
            [width - 1, 0],
            [width - 1, height - 1],
            [0, height - 1]
        ], dtype=np.float32)

        # Compute the perspective transform matrix
        matrix = cv2.getPerspectiveTransform(rect_corners, dst_corners)

        # Apply perspective warp to get the top-down view
        warped_image = cv2.warpPerspective(frame, matrix, (width, height))

        # Plot the inner rectangle on the flattened snapshot
        if {1, 2, 3, 4}.issubset(marker_positions.keys()):
            # Extract the inner rectangle corners dynamically
            inner_rect_corners = np.array([
                marker_positions[1][2],   # Bottom-right corner of marker 1
                marker_positions[2][3],  # Bottom-left corner of marker 2
                marker_positions[3][0],  # Top-left corner of marker 3
                marker_positions[4][1]    # Top-right corner of marker 4
            ], dtype=np.float32)

            # Draw the inner rectangle on the flattened snapshot
            transformed_inner_corners = cv2.perspectiveTransform(inner_rect_corners[None, :, :], matrix)
            transformed_inner_corners = np.int32(transformed_inner_corners[0])
            cv2.polylines(warped_image, [transformed_inner_corners], isClosed=True, color=(0, 0, 255), thickness=2)

        # Display the warped (flattened) image with the inner rectangle
        cv2.imshow('Flattened Snapshot', warped_image)
        # Define the output dimensions for the inner rectangle
        inner_width, inner_height = 200, 200  # Adjust as needed
        inner_dst_corners = np.array([
            [0, 0],
            [inner_width - 1, 0],
            [inner_width - 1, inner_height - 1],
            [0, inner_height - 1]
        ], dtype=np.float32)

        # Compute perspective transform to isolate the inner rectangle
        inner_matrix = cv2.getPerspectiveTransform(inner_rect_corners, inner_dst_corners)
        inner_rect_image = cv2.warpPerspective(frame, inner_matrix, (inner_width, inner_height))

        # Convert to a binary map for A* algorithm
        gray_inner_rect = cv2.cvtColor(inner_rect_image, cv2.COLOR_BGR2GRAY)
        _, binary_map = cv2.threshold(gray_inner_rect, 100, 255, cv2.THRESH_BINARY_INV)
        # Detect the ArUco marker with id=10 in the inner rectangle
        inner_corners, inner_ids, _ = cv2.aruco.detectMarkers(gray_inner_rect, aruco_dict, parameters=parameters)

        # Padding to hide regions of interests
        padding = 10

        ###   END
        # Makrer 5 = END (replace with 2 in binary map)
        # marker #5 -> 2 for binary map
        if inner_ids is not None and 5 in inner_ids:
            index = np.where(inner_ids == 5)[0][0]
            marker_corners = inner_corners[index][0]
            marker_corners_int = np.int32(marker_corners)

            # Calculate the center of the marker by averaging the corner points
            center_x = int(np.mean(marker_corners[:, 0]))
            center_y = int(np.mean(marker_corners[:, 1]))

            padded_corners = np.array([
                [marker_corners[0][0] - padding, marker_corners[0][1] - padding],
                [marker_corners[1][0] + padding, marker_corners[1][1] - padding],
                [marker_corners[2][0] + padding, marker_corners[2][1] + padding],
                [marker_corners[3][0] - padding, marker_corners[3][1] + padding],
            ], dtype=np.float32)

            #Mask the marker
            padded_mask = np.zeros_like(binary_map, dtype=np.uint8)
            padded_corners_int = np.int32(padded_corners)
            cv2.fillPoly(padded_mask, [padded_corners_int], 255)
            binary_map[padded_mask == 255] = 0

            # Red circle (END)
            cv2.circle(binary_map, (center_x, center_y), radius=5, color=2, thickness=-1)  # Set to 2 for goal

        ###   START
        # Makrer 6 = START (replace with 3 in binary map)
        # marker #6 -> 3 for binary map
        if inner_ids is not None and 6 in inner_ids:
            index = np.where(inner_ids == 6)[0][0]
            marker_corners = inner_corners[index][0]
            marker_corners_int = np.int32(marker_corners)

            # Calculate the center of the marker by averaging the corner points
            center_x = int(np.mean(marker_corners[:, 0]))
            center_y = int(np.mean(marker_corners[:, 1]))
            padded_corners = np.array([
                [marker_corners[0][0] - padding, marker_corners[0][1] - padding],
                [marker_corners[1][0] + padding, marker_corners[1][1] - padding],
                [marker_corners[2][0] + padding, marker_corners[2][1] + padding],
                [marker_corners[3][0] - padding, marker_corners[3][1] + padding],
            ], dtype=np.float32)

            #Mask the marker
            padded_mask = np.zeros_like(binary_map, dtype=np.uint8)
            padded_corners_int = np.int32(padded_corners)
            cv2.fillPoly(padded_mask, [padded_corners_int], 255)
            binary_map[padded_mask == 255] = 0

            # Green circle (START)
            cv2.circle(binary_map, (center_x, center_y), radius=5, color=3, thickness=-1)  # Set to 1 for start

        # Show Binary Map
        color_map = np.zeros((*binary_map.shape, 3), dtype=np.uint8)
        color_map[binary_map == 0] = (255, 255, 255)
        color_map[binary_map == 255] = (0, 0, 0)
        color_map[binary_map == 2] = (0, 0, 255)
        color_map[binary_map == 3] = (0, 255, 0)

        cv2.imshow('Binary Map', color_map)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        #Map in the format for path-finding
        grid_map = np.zeros_like(binary_map, dtype=int)

        ### Set regions of interests
        #Obstacles (1) 
        grid_map[binary_map == 255] = 1  
        # End (2)
        grid_map[binary_map == 2] = 2         
        # Start (3)
        grid_map[binary_map == 3] = 3   
        
        
        resized_grid_map = cv2.resize(grid_map, (50, 100), interpolation=cv2.INTER_NEAREST)

        current_dir = os.getcwd()
        folder_name = os.path.join(current_dir, "Maps")
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)

        np.savetxt(os.path.join(folder_name, "grid_map.txt"), grid_map, fmt='%d', delimiter=' ', comments='')
        np.savetxt(os.path.join(folder_name, "binary_map.txt"), binary_map, fmt='%d', delimiter=' ', comments='')
        np.savetxt(os.path.join(folder_name, "resized_grid_map.txt"), resized_grid_map, fmt='%d', delimiter=' ', comments='')


# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()